In [1]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2022_01.parquet')
ts_data

,pickup_hour,rides,pickup_location_id
0,2023-06-01 00:00:00,6,4
1,2023-06-01 01:00:00,1,4
2,2023-06-01 02:00:00,1,4
3,2023-06-01 03:00:00,0,4
4,2023-06-01 04:00:00,1,4
...,...,...,...
185755,2023-06-30 19:00:00,0,245
185756,2023-06-30 20:00:00,0,245
185757,2023-06-30 21:00:00,0,245
185758,2023-06-30 22:00:00,1,245


In [2]:
ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == 43, :].reset_index(drop=True)
ts_data_one_location.head(15)

,pickup_hour,rides,pickup_location_id
0,2023-06-01 00:00:00,9,43
1,2023-06-01 01:00:00,3,43
2,2023-06-01 02:00:00,0,43
3,2023-06-01 03:00:00,0,43
4,2023-06-01 04:00:00,3,43
5,2023-06-01 05:00:00,3,43
6,2023-06-01 06:00:00,14,43
7,2023-06-01 07:00:00,28,43
8,2023-06-01 08:00:00,57,43
9,2023-06-01 09:00:00,84,43


In [3]:
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int,
    step_size: int
    ) -> list:

        stop_position = len(data) - 1
        
        # Start the first sub-sequence at index position 0
        subseq_first_idx = 0
        subseq_mid_idx = n_features
        subseq_last_idx = n_features + 1
        indices = []
        
        while subseq_last_idx <= stop_position:
            indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))
            
            subseq_first_idx += step_size
            subseq_mid_idx += step_size
            subseq_last_idx += step_size

        return indices

In [4]:
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    ts_data_one_location,
    n_features,
    step_size
)
indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [5]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_17180\1674989729.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values


In [6]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(695, 24)
x=array([[  9.,   3.,   0., ..., 145.,  75.,  60.],
       [  3.,   0.,   0., ...,  75.,  60.,  45.],
       [  0.,   0.,   3., ...,  60.,  45.,   5.],
       ...,
       [ 60.,  39.,  42., ..., 143., 116., 113.],
       [ 39.,  42.,  26., ..., 116., 113.,  78.],
       [ 42.,  26.,  13., ..., 113.,  78.,  54.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2023-06-02 00:00:00'), Timestamp('2023-06-02 01:00:00'), Timestamp('2023-06-02 02:00:00'), Timestamp('2023-06-02 03:00:00'), Timestamp('2023-06-02 04:00:00')]


In [7]:
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,9.0,3.0,0.0,0.0,3.0,3.0,14.0,28.0,57.0,84.0,...,171.0,168.0,180.0,175.0,150.0,158.0,173.0,145.0,75.0,60.0
1,3.0,0.0,0.0,3.0,3.0,14.0,28.0,57.0,84.0,84.0,...,168.0,180.0,175.0,150.0,158.0,173.0,145.0,75.0,60.0,45.0
2,0.0,0.0,3.0,3.0,14.0,28.0,57.0,84.0,84.0,129.0,...,180.0,175.0,150.0,158.0,173.0,145.0,75.0,60.0,45.0,5.0
3,0.0,3.0,3.0,14.0,28.0,57.0,84.0,84.0,129.0,176.0,...,175.0,150.0,158.0,173.0,145.0,75.0,60.0,45.0,5.0,2.0
4,3.0,3.0,14.0,28.0,57.0,84.0,84.0,129.0,176.0,141.0,...,150.0,158.0,173.0,145.0,75.0,60.0,45.0,5.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,111.0,93.0,60.0,39.0,42.0,26.0,13.0,2.0,5.0,0.0,...,30.0,55.0,99.0,83.0,125.0,130.0,177.0,153.0,158.0,143.0
691,93.0,60.0,39.0,42.0,26.0,13.0,2.0,5.0,0.0,4.0,...,55.0,99.0,83.0,125.0,130.0,177.0,153.0,158.0,143.0,116.0
692,60.0,39.0,42.0,26.0,13.0,2.0,5.0,0.0,4.0,2.0,...,99.0,83.0,125.0,130.0,177.0,153.0,158.0,143.0,116.0,113.0
693,39.0,42.0,26.0,13.0,2.0,5.0,0.0,4.0,2.0,8.0,...,83.0,125.0,130.0,177.0,153.0,158.0,143.0,116.0,113.0,78.0


In [8]:
targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])
targets_one_location

,target_rides_next_hour
0,45.0
1,5.0
2,2.0
3,2.0
4,1.0
...,...
690,116.0
691,113.0
692,78.0
693,54.0


In [9]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame,
    input_seq_len: int,
    step_size: int
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train Supervised ML models
    """
    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    
    for location_id in tqdm(location_ids):
        
        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id, 
            ['pickup_hour', 'rides']
        ]

        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [10]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

  0%|          | 0/258 [00:00<?, ?it/s]C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_17180\1689250040.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_17180\1689250040.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_17180\1689250040.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your arr

features.shape=(5934, 170)
targets.shape=(5934,)
